In [1]:
import pandas as pd
df = pd.read_csv('TestHelp.csv')

In [ ]:
#I used AI to help with the cleaning for this part.  It was on the days of the mock presentations when Mr. Hussain
#suggested a different way of analysis.  I didn't have the information I required which was the number of Unique establishments
#of a brand.  
#So, I tried to get it to create one for me using multipe other columns in the dataset.
#I tried to use the name/lisence#/Latitude/Longitide/Address/Zip to figure out if it is a unique establishment of a brand or not.
#I had it try to generate two columns for me that I would use. One is the Brand Unique Count which would tell me how many unique (SUBWAYS for example)
#are in the dataset, so it would put that same number for all different (SUBWAY's).
#And the other is Brand Instance which would show up as (SUBWAY1/SUBWAY2/....etc).

#I only used this for one visual, but I'm unsure if it was the correct way to go about it.

In [1]:
import pandas as pd
import numpy as np
import re

# ---------------- helpers ----------------
def norm_text(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.upper()
    s = s.str.replace(r"[^\w\s]", " ", regex=True)  # remove punctuation
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

SUITE_PAT = r"\b(STE|SUITE|UNIT|APT|RM|ROOM|FL|FLOOR|#)\s*\w+\b"

def canon_address(s: pd.Series) -> pd.Series:
    s = norm_text(s)
    s = s.str.replace(SUITE_PAT, "", regex=True)              # drop suites/units
    # street type compaction (tweak as you like)
    s = s.str.replace(r"\bAVENUE\b", "AVE", regex=True)
    s = s.str.replace(r"\bSTREET\b", "ST", regex=True)
    s = s.str.replace(r"\bBOULEVARD\b", "BLVD", regex=True)
    s = s.str.replace(r"\bROAD\b", "RD", regex=True)
    s = s.str.replace(r"\bDRIVE\b", "DR", regex=True)
    s = s.str.replace(r"\bHIGHWAY\b", "HWY", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

def canon_brand(dba: pd.Series) -> pd.Series:
    """Normalize brand names; collapse trailing store numbers/variants."""
    s = norm_text(dba)
    # common ‘brand + number’ patterns → keep just the brand word(s)
    # e.g., "SUBWAY #12345", "MCDONALD'S 123", "STARBUCKS - O'HARE T2"
    s = s.str.replace(r"\s#?\d+\b.*$", "", regex=True)  # drop trailing numbers and anything after
    s = s.str.replace(r"\s+-\s+.*$", "", regex=True)    # drop text after hyphen
    s = s.str.strip()
    return s

# --------------- main --------------------
# Load your file
df = pd.read_csv("TestHelp.csv", low_memory=False)

# Ensure needed columns exist (change names if your file uses different ones)
REQ = ["DBA Name", "Address", "Zip", "License #", "Latitude", "Longitude"]
missing = [c for c in REQ if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Canonical fields
df["_BRAND"] = canon_brand(df["DBA Name"])
df["_ADDR"]  = canon_address(df["Address"])
df["_ZIP5"]  = (
    df["Zip"].astype(str).str.extract(r"(\d{5})", expand=False).str.zfill(5)
)

# License handling
lic = pd.to_numeric(df["License #"], errors="coerce").fillna(0).astype(int)
df["_HAS_LIC"] = lic > 0
df["_LIC_STR"] = lic.astype(str)

# Coarse lat/lon key (helps merge minor jitter; 3 decimals ~110m)
lat = pd.to_numeric(df["Latitude"], errors="coerce").round(3)
lon = pd.to_numeric(df["Longitude"], errors="coerce").round(3)
df["_LL3"] = np.where(lat.notna() & lon.notna(), lat.astype(str) + "_" + lon.astype(str), np.nan)

# Preferred, stable location key:
# 1) Canonical Address+ZIP (collapses license churn at same site)
# 2) License #
# 3) Lat/Lon (coarse)
df["_ADDRZIP"] = np.where(df["_ADDR"].notna() & df["_ZIP5"].notna(),
                          df["_ADDR"] + "_" + df["_ZIP5"], np.nan)

df["Loc_Key"] = np.select(
    [df["_ADDRZIP"].notna(),
     df["_HAS_LIC"],
     df["_LL3"].notna()],
    [df["_ADDRZIP"],
     "LIC_" + df["_LIC_STR"],
     "LL3_" + df["_LL3"]],
    default=np.nan
)

# Drop rows that can’t be located at all (optional)
# df = df[df["Loc_Key"].notna()].copy()

# For each brand, enumerate its unique Loc_Key as BRAND1, BRAND2, ...
# 1) make a table of brand+location uniques
brand_loc = (df.loc[df["Loc_Key"].notna(), ["_BRAND", "Loc_Key"]]
               .drop_duplicates()
               .sort_values(["_BRAND", "Loc_Key"])
               .reset_index(drop=True))

# 2) assign instance numbers within each brand
brand_loc["n"] = brand_loc.groupby("_BRAND").cumcount() + 1
brand_loc["Brand_Instance"] = np.where(
    brand_loc["_BRAND"].notna(),
    brand_loc["_BRAND"].str.replace(r"\s+", "", regex=True) + brand_loc["n"].astype(str),
    np.nan
)

# 3) brand unique counts
brand_counts = (brand_loc.groupby("_BRAND")["Loc_Key"]
                .nunique()
                .reset_index(name="Brand_Unique_Count"))

# 4) bring back to the row level
df = df.merge(brand_loc[["_BRAND","Loc_Key","Brand_Instance"]],
              on=["_BRAND","Loc_Key"], how="left")

df = df.merge(brand_counts, on="_BRAND", how="left")

# Clean up / final columns
df.rename(columns={
    "_BRAND":"Brand_Canon"
}, inplace=True)

# Optional: set non-brand rows’ counts to 0 (or leave NaN)
df["Brand_Unique_Count"] = df["Brand_Unique_Count"].fillna(0).astype(int)

# Save (or continue analysis)
df.to_csv("Inspections_with_brand_instances.csv", index=False)
print("Saved: Inspections_with_brand_instances.csv")

# Quick sanity checks (top brands by unique locations)
print("\nTop brands by unique locations:")
print(df.dropna(subset=["Brand_Instance"])
        .groupby("Brand_Canon")["Brand_Instance"]
        .nunique().sort_values(ascending=False).head(15))

# Example: see instances for a specific brand, e.g., SUBWAY
# print(df[df["Brand_Canon"]=="SUBWAY"][["DBA Name","Address","Zip","Brand_Instance"]]
#         .drop_duplicates().head(20))


Saved: Inspections_with_brand_instances.csv

Top brands by unique locations:
Brand_Canon
SUBWAY                          257
STARBUCKS COFFEE                173
DUNKIN DONUTS                   167
WALGREENS                       147
7 ELEVEN                        109
MCDONALD S                       72
CVS PHARMACY                     71
FAMILY DOLLAR                    62
BURGER KING                      58
DOLLAR TREE                      57
DUNKIN DONUTS BASKIN ROBBINS     56
CITGO                            55
MCDONALDS                        42
JEWEL FOOD STORE                 40
CHIPOTLE MEXICAN GRILL           39
Name: Brand_Instance, dtype: int64


In [10]:
df.head()

Inspection ID                  DBA Name  License #  \
0        2622439     SMART LEARNING CENTER  2231591.0   
1        2622414  GENERATION LIQUOR DEPORT  3020171.0   
2        2622443           FELD RESTAURANT  2948639.0   
3        2622445                   EZBACHI  3030788.0   
4        2622418    ROSA ISELA TACOS Y MAS  2470637.0   

                  Facility Type           Risk                 Address  \
0  Children's Services Facility  Risk 1 (High)  2980-2988 S ARCHER AVE   
1                        Liquor   Risk 3 (Low)       3900 W MADISON ST   
2                    Restaurant  Risk 1 (High)      2018 W CHICAGO AVE   
3          Mobile Food Preparer  Risk 1 (High)        135 N KEDZIE AVE   
4                    Restaurant  Risk 1 (High)    2551 W FULLERTON AVE   

       Zip Inspection Date        Inspection Type Results  ...  \
0  60608.0      2025-08-19  License Re-Inspection    Pass  ...   
1  60624.0      2025-08-19                License    Pass  ...   
2  60622.0      2025-08-19                Canvass    Pass  ...   
3  60612.0      2025-08-19  License Re-Inspection    Fail  ...   
4  60647.0      2025-08-19                Canvass    Fail  ...   

                Brand_Canon                   _ADDR  _ZIP5 _HAS_LIC  _LIC_STR  \
0     SMART LEARNING CENTER  2980 2988 S ARCHER AVE  60608     True   2231591   
1  GENERATION LIQUOR DEPORT       3900 W MADISON ST  60624     True   3020171   
2           FELD RESTAURANT      2018 W CHICAGO AVE  60622     True   2948639   
3                   EZBACHI        135 N KEDZIE AVE  60612     True   3030788   
4    ROSA ISELA TACOS Y MAS    2551 W FULLERTON AVE  60647     True   2470637   

             _LL3                      _ADDRZIP                       Loc_Key  \
0   41.841_-87.66  2980 2988 S ARCHER AVE_60608  2980 2988 S ARCHER AVE_60608   
1  41.881_-87.723       3900 W MADISON ST_60624       3900 W MADISON ST_60624   
2  41.896_-87.678      2018 W CHICAGO AVE_60622      2018 W CHICAGO AVE_60622   
3  41.884_-87.706        135 N KEDZIE AVE_60612        135 N KEDZIE AVE_60612   
4  41.925_-87.692    2551 W FULLERTON AVE_60647    2551 W FULLERTON AVE_60647   

            Brand_Instance Brand_Unique_Count  
0     SMARTLEARNINGCENTER1                  1  
1  GENERATIONLIQUORDEPORT1                  1  
2          FELDRESTAURANT1                  1  
3                 EZBACHI1                  1  
4      ROSAISELATACOSYMAS1                  1  

[5 rows x 32 columns]